# Crossvalidation Metrics

Script to pull in results of each cemetary model run `"{cemetary}_crossval_results_updated_v2.csv"` and create a output dataframe of metrics based on a threshold of 0.3

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pwd

/gpfs/data/bershteynlab/SatelliteImagery/weitzk01


In [3]:
cemetaries = ["DAR_UNLABLED_1", "DAR_UNLABLED_2", "DAR_UNLABLED_3", "DAR_UNLABLED_4", "DAR_UNLABLED_5", "DAR_UNLABLED_6", "DAR_UNLABLED_7", "Ismail Cementary", "Ismaili Cemetery", "Karume Cemetery", "Mburahati", "Mianzini Graveyard", "Ubungo", "Msasani", "Mikocheni"]

In [4]:
# Create file of grave counts
data = []
for c in cemetaries:
    temp_df = pd.read_csv(f"{c}_crossval_results_updated_v2.csv")
    
    temp_df['Cemetary'] = c

    # Sum across cemetary
    summed_df = temp_df.groupby(['ModelName', 'Cemetary', 'ConfidenceThreshold']).aggregate({"PredictedCount": sum, "ActualCount": sum})

    # Get best Confidence threshold
    
    data.append(temp_df)

ungrouped_df = pd.concat(data)
deduped = ungrouped_df[['Cemetary', 'FileName', 'ActualCount']].drop_duplicates()
deduped.to_csv("Cemetary_Actual_Counts_Updated.csv", index=False)

In [5]:
# Calculate metrics at .3 threshold
data = []
for c in cemetaries:
    temp_df = pd.read_csv(f"{c}_crossval_results_updated_v2.csv")
    
    temp_df['Cemetary'] = c

    # Sum across cemetary
    summed_df = temp_df.groupby(['ModelName', 'Cemetary', 'ConfidenceThreshold']).aggregate({"PredictedCount": sum, "ActualCount": sum})
    
    data.append(summed_df)

df = pd.concat(data)

ungroup_df = df.reset_index()

final_df = ungroup_df[ungroup_df['ConfidenceThreshold'] == 0.3]

# Calculate MAPE
final_df['MAPE'] = np.abs((final_df['ActualCount'] - final_df['PredictedCount']) / final_df['ActualCount']) * 100

# Calculate MAE
final_df['MAE'] = np.abs(final_df['ActualCount'] - final_df['PredictedCount'])

# Calculate RMSE
final_df['RMSE'] = np.sqrt(((final_df['ActualCount'] - final_df['PredictedCount']) ** 2))

# Display the updated DataFrame
final_df.head()

/tmp/ipykernel_3560061/1602990756.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['MAPE'] = np.abs((final_df['ActualCount'] - final_df['PredictedCount']) / final_df['ActualCount']) * 100
/tmp/ipykernel_3560061/1602990756.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['MAE'] = np.abs(final_df['ActualCount'] - final_df['PredictedCount'])
/tmp/ipykernel_3560061/1602990756.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

,ModelName,Cemetary,ConfidenceThreshold,PredictedCount,ActualCount,MAPE,MAE,RMSE
2,MMRotate,DAR_UNLABLED_1,0.3,399,562,29.003559,163,163.0
11,MMRotate,DAR_UNLABLED_2,0.3,467,485,3.711340,18,18.0
20,MMRotate,DAR_UNLABLED_3,0.3,69,64,7.812500,5,5.0
29,MMRotate,DAR_UNLABLED_4,0.3,183,268,31.716418,85,85.0
38,MMRotate,DAR_UNLABLED_5,0.3,233,284,17.957746,51,51.0


In [6]:
final_df.sort_values('Cemetary')

,ModelName,Cemetary,ConfidenceThreshold,PredictedCount,ActualCount,MAPE,MAE,RMSE
2,MMRotate,DAR_UNLABLED_1,0.3,399,562,29.003559,163,163.0
11,MMRotate,DAR_UNLABLED_2,0.3,467,485,3.711340,18,18.0
20,MMRotate,DAR_UNLABLED_3,0.3,69,64,7.812500,5,5.0
29,MMRotate,DAR_UNLABLED_4,0.3,183,268,31.716418,85,85.0
38,MMRotate,DAR_UNLABLED_5,0.3,233,284,17.957746,51,51.0
47,MMRotate,DAR_UNLABLED_6,0.3,51,41,24.390244,10,10.0
56,MMRotate,DAR_UNLABLED_7,0.3,237,195,21.538462,42,42.0
65,MMRotate,Ismail Cementary,0.3,289,317,8.832808,28,28.0
74,MMRotate,Ismaili Cemetery,0.3,1437,1818,20.957096,381,381.0
83,MMRotate,Karume Cemetery,0.3,190,133,42.857143,57,57.0


In [8]:
final_df['MAPE'].mean()

23.17142942320829

In [10]:
final_df['MAPE'].describe()

count    15.000000
mean     23.171429
std      18.438805
min       1.853871
25%       8.322654
50%      20.957096
75%      30.359988
max      66.153846
Name: MAPE, dtype: float64

In [11]:
final_df['MAPE'].median()

20.957095709570957

In [12]:
final_df['MAE'].describe()

count     15.000000
mean     102.466667
std      164.313666
min        5.000000
25%       17.500000
50%       43.000000
75%       71.000000
max      586.000000
Name: MAE, dtype: float64

In [13]:
final_df['MAE'].median()

43.0

In [7]:
final_df.to_csv("MMRotate_Final_Metrics_Update_lower_threshold.csv", index=False)